In [1]:
import plotly.graph_objects as go

from apollo.calculations.average_true_range import AverageTrueRangeCalculator
from apollo.calculations.elliot_waves_calculator import ElliotWavesCalculator
from apollo.providers.price_data_enhancer import PriceDataEnhancer
from apollo.providers.price_data_provider import PriceDataProvider
from apollo.settings import FREQUENCY, TICKER
from apollo.utils.common import to_default_date_string

In [2]:
start_date = "1995-01-01"
end_date = "2024-09-01"

window_size = 5

price_data_provider = PriceDataProvider()
price_data_enhancer = PriceDataEnhancer()

dataframe = price_data_provider.get_price_data(
    ticker=str(TICKER),
    frequency=str(FREQUENCY),
    start_date=start_date,
    end_date=end_date,
    max_period=False,
)

dataframe = price_data_enhancer.enhance_price_data(
    price_dataframe=dataframe,
    additional_data_enhancers=["VIX", "SP500 Futures"],
)

dataframe["prev_close"] = dataframe["adj close"].shift(1)

atr_calculator = AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=window_size,
)
atr_calculator.calculate_average_true_range()

ew_calculator = ElliotWavesCalculator(
    dataframe=dataframe,
    window_size=window_size,
    fast_oscillator_period=5,
    slow_oscillator_period=10,
)
ew_calculator.calculate_elliot_waves()

dataframe = dataframe.loc["2007-06-01":"2008-01-01"]

dataframe

,ticker,adj close,adj high,adj low,adj open,adj volume,close,high,low,open,...,prev_close,tr,atr,high_low_avg,fast_hla_sma,slow_hla_sma,ewo,ewt,prev_ewt,ew
date,,,,,,,,,,,,,,,,,,,,,
2007-06-01,SPY,110.408318,110.637613,109.999870,110.265007,7.722541e+07,154.080002,154.399994,153.509995,153.880005,...,109.863739,0.773874,0.901653,110.318741,109.415877,109.364282,0.051595,0.0,-1.0,0.0
2007-06-04,SPY,110.422646,110.630445,109.978368,110.021360,5.589836e+07,154.100006,154.389999,153.479996,153.539993,...,110.408318,0.652077,0.851738,110.304406,109.750506,109.487529,0.262977,1.0,0.0,3.0
2007-06-05,SPY,109.985603,110.279387,109.534164,110.164745,9.094496e+07,153.490005,153.899994,152.860001,153.740005,...,110.422646,0.888482,0.859087,109.906775,109.951875,109.524442,0.427434,0.0,1.0,0.0
2007-06-06,SPY,108.803223,109.598610,108.738734,109.534122,1.175860e+08,151.839996,152.949997,151.750000,152.860001,...,109.985603,1.246869,0.936643,109.168672,109.938978,109.494345,0.444633,1.0,0.0,0.0
2007-06-07,SPY,106.839828,109.276145,106.811160,108.602572,1.665401e+08,149.100006,152.500000,149.059998,151.559998,...,108.803223,2.464985,1.242311,108.043652,109.548450,109.340285,0.208164,-1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-12-24,SPY,108.475998,108.657724,107.930819,108.177975,3.314788e+07,149.229996,149.479996,148.479996,148.820007,...,107.676445,0.981279,1.539611,108.294272,106.358128,106.841867,-0.483739,1.0,0.0,4.0
2007-12-26,SPY,108.708679,108.803169,107.945426,108.054458,4.877033e+07,149.550003,149.679993,148.500000,148.649994,...,108.475998,0.857743,1.403238,108.374298,107.032221,106.807258,0.224964,0.0,1.0,5.0
2007-12-27,SPY,107.342064,108.330655,107.087654,108.323390,8.939602e+07,147.669998,149.029999,147.320007,149.020004,...,108.708679,1.621025,1.446795,107.709154,107.472780,106.769400,0.703380,1.0,0.0,3.0


In [3]:
value_1 = "adj close"
value_2 = "ew"
value_3 = "ewo"
value_4 = "adj low"

x = dataframe.index.to_numpy()

y1 = dataframe[value_1].to_numpy()
y2 = dataframe[value_2].to_numpy()
y3 = dataframe[value_3].to_numpy()
y4 = dataframe[value_4].to_numpy()

trace1 = go.Scatter(x=x, y=y1, name=value_1, yaxis="y1")
trace2 = go.Scatter(x=x, y=y2, name=value_2, yaxis="y2")
trace3 = go.Scatter(x=x, y=y3, name=value_3, yaxis="y2")
trace4 = go.Scatter(x=x, y=y4, name=value_4, yaxis="y2")

# Plot title
title = (
    f"{dataframe.iloc[0]["ticker"]}"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(
    data=[
        trace1,
        trace2,
        # trace3,
        # trace4,
    ],
    layout=layout,
)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)